<a href="https://colab.research.google.com/github/ehersch/MNIST_Demo/blob/main/mnist_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F

In [15]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


In [20]:
trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transforms.ToTensor())
testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transforms.ToTensor())

In [9]:
# Old image w/o using a transform (just loads in raw image)
# trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=none)
trainset[0][0]

In [38]:
class Model(nn.Module):
    def __init__(self, hidden):
        super(Model, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 3, 1)
        self.conv2 = nn.Conv2d(16, 32, 3, 1)
        self.fc1 = nn.Linear(32 * 5 * 5, hidden)

        self.pool = nn.MaxPool2d(2,2)
        self.fc2 = nn.Linear(hidden, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.pool(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = self.pool(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x

In [39]:
cnn_model = Model(100)
optimizer = torch.optim.Adam(cnn_model.parameters(), lr=0.001)

In [40]:
def train(model, train_loader, optimizer, epochs):
    model.train()
    for epoch in range(epochs):
        print(f"Epoch {epoch+1}\n-------------------------------")

        for batch, (X, y) in enumerate(train_loader):
            X, y = X.to(device), y.to(device)
            optimizer.zero_grad()
            y_pred = model(X)
            loss = F.cross_entropy(y_pred, y)
            loss.backward()
            optimizer.step()
            if batch % 100 == 0:
                loss, current = loss.item(), batch * len(X)
                print(f"loss: {loss:>7f}  [{current:>5d}/{len(train_loader.dataset):>5d}]")

In [41]:
train_loader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

In [42]:
train(cnn_model, train_loader=train_loader, optimizer=optimizer, epochs=7)

Epoch 1
-------------------------------
loss: 2.304642  [    0/60000]
loss: 0.390386  [ 6400/60000]
loss: 0.258242  [12800/60000]
loss: 0.351176  [19200/60000]
loss: 0.116036  [25600/60000]
loss: 0.104244  [32000/60000]
loss: 0.083680  [38400/60000]
loss: 0.105745  [44800/60000]
loss: 0.017518  [51200/60000]
loss: 0.050139  [57600/60000]
Epoch 2
-------------------------------
loss: 0.051522  [    0/60000]
loss: 0.186481  [ 6400/60000]
loss: 0.167308  [12800/60000]
loss: 0.064514  [19200/60000]
loss: 0.009418  [25600/60000]
loss: 0.206538  [32000/60000]
loss: 0.090565  [38400/60000]
loss: 0.225703  [44800/60000]
loss: 0.030005  [51200/60000]
loss: 0.018085  [57600/60000]
Epoch 3
-------------------------------
loss: 0.039865  [    0/60000]
loss: 0.073914  [ 6400/60000]
loss: 0.007287  [12800/60000]
loss: 0.047908  [19200/60000]
loss: 0.048691  [25600/60000]
loss: 0.027041  [32000/60000]
loss: 0.011033  [38400/60000]
loss: 0.006456  [44800/60000]
loss: 0.064886  [51200/60000]
loss: 0.01

In [43]:
test_loader = torch.utils.data.DataLoader(trainset, batch_size=1, shuffle=True)

In [44]:
def test(model, test_loader):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for X, y in test_loader:
            X, y = X.to(device), y.to(device)

            outputs = model(X)                 # (batch_size, num_classes)
            preds = outputs.argmax(dim=1)      # predicted class indices

            correct += (preds == y).sum().item()
            total += y.size(0)

    print(f"Accuracy: {correct / total:.4f}")


In [45]:
test(cnn_model, test_loader)

Accuracy: 0.9975
